# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,70.23,67,82,5.46,PN,1717024208
1,1,tazovsky,67.4667,78.7000,16.23,95,74,8.28,RU,1717024642
2,2,dalandzadgad,43.5708,104.4250,57.63,39,26,5.26,MN,1717024642
3,3,margaret river,-33.9500,115.0667,58.62,66,98,14.32,AU,1717024392
4,4,puerto aysen,-45.4000,-72.7000,39.67,99,100,3.40,CL,1717024590


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df['Max Temp'] >= 70) & (city_data_df['Max Temp'] <= 80) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,cerano,20.1167,-101.3833,73.17,29,77,3.96,MX,1717024643
32,32,albany,42.6001,-73.9662,72.25,47,87,1.01,US,1717024581
35,35,formosa do rio preto,-11.0483,-45.1931,76.21,41,79,2.98,BR,1717024645
63,63,kaka,37.3482,59.6143,73.17,24,0,6.13,TM,1717024647
64,64,port alfred,-33.5906,26.8910,70.18,43,0,3.00,ZA,1717024647


In [5]:
ideal_df.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,2.000000e+01
mean,227.500000,18.696000,4.319735,74.989000,33.100000,35.150000,7.537000,1.717025e+09
std,161.151612,31.623972,87.298473,3.311346,12.657014,33.994233,5.256202,5.783497e+01
min,19.000000,-33.590600,-102.467700,70.180000,1.000000,0.000000,1.010000,1.717025e+09
25%,65.500000,-11.382650,-79.083075,72.217500,27.750000,0.750000,3.720000,1.717025e+09
50%,261.000000,28.628300,20.404700,74.090000,37.000000,24.500000,6.150000,1.717025e+09
75%,339.250000,43.909200,55.416700,77.882500,43.000000,72.000000,9.572500,1.717025e+09
max,499.000000,62.668300,164.865800,79.990000,48.000000,90.000000,20.710000,1.717025e+09


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns = ["City","Country","Lat","Lng","Humidity"]
hotel_df = ideal_df[columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
19,cerano,MX,20.1167,-101.3833,29,
32,albany,US,42.6001,-73.9662,47,
35,formosa do rio preto,BR,-11.0483,-45.1931,41,
63,kaka,TM,37.3482,59.6143,24,
64,port alfred,ZA,-33.5906,26.8910,43,
66,mitchell,US,43.7094,-98.0298,37,
81,murzuq,LY,25.9155,13.9184,31,
97,kenora,CA,49.8167,-94.4337,37,
156,najran,SA,17.4924,44.1277,18,
228,kone,NC,-21.0595,164.8658,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to indicate the start of the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lon},{lat},{radius}",
        "apiKey": geoapify_key,
        "bias": f"proximity:{lon},{lat}"
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data from hotel_df
hotel_df

Starting hotel search
cerano - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
formosa do rio preto - nearest hotel: Pousada Universo - Formosa do Rio Preto BA
kaka - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
mitchell - nearest hotel: Quality Inn
murzuq - nearest hotel: فندق باريس - مرزق
kenora - nearest hotel: Travelodge
najran - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
taguatinga - nearest hotel: No hotel found
borogontsy - nearest hotel: No hotel found
fort-shevchenko - nearest hotel: Akku
jaicos - nearest hotel: Apart-Hotel Restô Da Roça
flin flon - nearest hotel: Copperbelt Hotel
ardakan - nearest hotel: هتل رستوران باغ سرای اردکان
east london - nearest hotel: No hotel found
sarina - nearest hotel: Tandara Hotel
zhob - nearest hotel: No hotel found
yorkton - nearest hotel: Holiday Inn Express


,City,Country,Lat,Lng,Humidity,Hotel Name
19,cerano,MX,20.1167,-101.3833,29,No hotel found
32,albany,US,42.6001,-73.9662,47,No hotel found
35,formosa do rio preto,BR,-11.0483,-45.1931,41,Pousada Universo - Formosa do Rio Preto BA
63,kaka,TM,37.3482,59.6143,24,No hotel found
64,port alfred,ZA,-33.5906,26.8910,43,No hotel found
66,mitchell,US,43.7094,-98.0298,37,Quality Inn
81,murzuq,LY,25.9155,13.9184,31,فندق باريس - مرزق
97,kenora,CA,49.8167,-94.4337,37,Travelodge
156,najran,SA,17.4924,44.1277,18,No hotel found
228,kone,NC,-21.0595,164.8658,41,Pacifik Appartels


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    hover_cols = ["Country","Hotel Name"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Country,Hotel Name)